In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import quote
import requests
import re          #정규표현식을 위해
from tqdm.notebook import tqdm

In [ ]:
#http://book.interpark.com/display/collectlist.do?_method=BestsellerHourNew201605&bestTp=1&dispNo=028&smid1=s_menu&smid2=bestseller

In [2]:
base_url='http://book.interpark.com/'
sub_url='display/collectlist.do?_method=BestsellerHourNew201605&bestTp=1&dispNo=028&smid1=s_menu&smid2=bestseller'
url = base_url+sub_url
res = requests.get(url)
soup = BeautifulSoup(res.text,'html.parser')

In [ ]:
soup

In [18]:
lis = soup.select('.rankBestContentList > ol > li')
len(lis)

15

In [19]:
li = lis[0]
href=li.find('a')['href']
href

'/product/BookDisplay.do?_method=detail&sc.shopNo=0000400000&sc.prdNo=245785547&sc.saNo=003003001&bid1=Best_zone&bid2=LiveRanking&bid3=PRD&bid4=001'

In [33]:
base_url+href

'http://book.interpark.com//product/BookDisplay.do?_method=detail&sc.shopNo=0000400000&sc.prdNo=245785547&sc.saNo=003003001&bid1=Best_zone&bid2=LiveRanking&bid3=PRD&bid4=001'

In [34]:
base_url='http://book.interpark.com'

In [96]:
res=requests.get(base_url+href)
book_soup=BeautifulSoup(res.text,'html.parser')
book_soup


<!DOCTYPE html>

<html lang="ko">
<head>
<title>싸니까 믿으니까 인터파크도서</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="인터파크도서, 인터넷 서점, eBook, 비스킷탭, 음반, DVD, 중고책, 중고책서점, 신간도서, 국내도서, 외국도서, 추천도서" name="keywords"/>
<meta content="문학은 인터파크 도서!" name="description"/>
<meta content="no-cache" http-equiv="Cache-Control">
<meta content="0" http-equiv="Expires">
<meta content="no-cache" http-equiv="Pragma">
<link href="https://sslimage.interpark.com/interpark.ico" rel="shortcut icon"/>
<meta content="no" http-equiv="imagetoolbar">
<script type="text/javascript">
        var isFancyAlert = false;    // IMFS에서는 msgAlert 기능을 비활성화시킴
        var imfsSSLDomain = "https://book.interpark.com"; // com.interpark.util.web.ServletUtils.IMFS_URL_SSL 값을 사용하였으나, Fix 처리 한듯.
    </script>
<script src="/static/js/common_ssl.js" type="text/javascript"></script>
<script src="/static/js/CookieUtil.js" type="text/java

In [97]:
info=book_soup.select_one('.titleWrap')
title = info.find('h2').get_text()
title

'기분을 관리하면 인생이 관리된다 : 김다슬 에세이'

In [98]:
info2=book_soup.select_one('.optionRight_wrap')
book_list=info2.select('.bookInfoBox > ul > li')
#book_list[0]
author = book_list[0].find('a').get_text().strip()
author

'김다슬'

In [99]:
book_list[1]

<li>출판사 : <a href="#" id="hdelvMafcEntrNm" onclick='fn_wiseLogRedirect("infor","basic_inform","publisher","http://bsearch.interpark.com/dsearch/book.jsp?sch=all&amp;query=클라우디아","_blank");return false;'>클라우디아</a></li>

In [64]:
book_list[3].get_text().split(':')[1]

' 2021년 11월 29일'

In [ ]:
book_list[3].find('span').get_text().strip().split(':')[1]

In [84]:
info3=book_soup.select_one('.totalTab')
book_list2=info3.select('.tabBox_1')
price=book_list2[0].find('p').get_text().strip().split('원')[0]
price

'15,120'

In [ ]:
line=[]
for li in lis:
  list = li
  href=list.find('a')['href']
  res=requests.get(base_url+href)
  book_soup=BeautifulSoup(res.text,'html.parser')
  info=book_soup.select_one('.titleWrap')
  title = info.find('h2').get_text()
  info2=book_soup.select_one('.optionRight_wrap')
  book_list=info2.select('.bookInfoBox > ul > li')
  author = book_list[0].find('a').get_text().strip()
  if len(book_list) == 5:
      translator = '-'
      date = book_list[2].get_text().split(':')[1]
      page = book_list[3].get_text().split(':')[1]
  elif len(book_list) == 7:
      translator = '-'
      date = book_list[4].get_text().split(':')[1]
      page = book_list[5].get_text().split(':')[1]
  else:
      translator = book_list[1].find('a').get_text().strip()
      date = book_list[3].get_text().split(':')[1]
      page = book_list[4].get_text().split(':')[1]
  info3=book_soup.select_one('.totalTab')
  book_list2=info3.select('.tabBox_1')
  price=book_list2[0].find('p').get_text().strip().split('원')[0]
  line.append({'제목':title,'저자':author,'번역':translator,'출시일':date,'쪽수':page,'가격':price})
line

In [110]:
df=pd.DataFrame(line)
df

,제목,저자,번역,출시일,쪽수,가격
0,5년 후 나에게 Q&A a Day(2022 메리골드 Edition) [양장],포터 스타일,정지현,2021년 11월 29일,368,"15,120"
1,나를 살리는 논어 한마디 : 거친 물결에 흔들리는 삶을 잡아줄 공자의 명쾌한 해답,판덩,이서연,2022년 05월 30일,320,"16,020"
2,흔한남매 10,흔한남매,-,2022년 04월 28일,192,"12,150"
3,SQL 자격검정 실전문제 : 국가공인 SQL전문가 국가공인 SQL개발자,한국데이터진흥원,-,2021년 10월 15일,279,"17,280"
4,작별인사 : 김영하 장편소설,김영하,-,2022년 05월 02일,308,"12,600"
5,"운명을 바꾸는 부동산 투자 수업: 실전편 : 내 집 마련부터 실전 아파트 투자까지,...",정태익,-,2022년 03월 22일,300,"15,120"
6,불편한 편의점(40만부 기념 벚꽃 에디션) : 김호연 장편소설,김호연,-,2021년 04월 20일,268,"12,600"
7,쥬비스 미라클 : 쥬비스 다이어트는 어떻게 신화가 되었나?,조성경,-,2022년 03월 28일,284,"14,400"
8,"운명을 바꾸는 부동산 투자 수업: 기초편 : 내 집 마련부터 실전 아파트 투자까지,...",정태익,-,2022년 03월 22일,300,"15,120"
9,나에게 고맙다(30만 부 기념 전면 개정판) : 전승환 에세이,전승환,-,2022년 02월 18일,288,"14,400"


In [111]:
df.to_csv('인터파크 베스크셀러.csv',index=False)

In [112]:
from tqdm.notebook import tqdm

In [ ]:
line2=[]
for page in tqdm(range(1)):
  for li in lis:
    list = li
    href=list.find('a')['href']
    res=requests.get(base_url+href)
    book_soup=BeautifulSoup(res.text,'html.parser')
    info=book_soup.select_one('.titleWrap')
    title = info.find('h2').get_text()
    info2=book_soup.select_one('.optionRight_wrap')
    book_list=info2.select('.bookInfoBox > ul > li')
    author = book_list[0].find('a').get_text().strip()
    if len(book_list) == 5:
        translator = '-'
        date = book_list[2].get_text().split(':')[1]
        page = book_list[3].get_text().split(':')[1]
    elif len(book_list) == 7:
        translator = '-'
        date = book_list[4].get_text().split(':')[1]
        page = book_list[5].get_text().split(':')[1]
    else:
        translator = book_list[1].find('a').get_text().strip()
        date = book_list[3].get_text().split(':')[1]
        page = book_list[4].get_text().split(':')[1]
    info3=book_soup.select_one('.totalTab')
    book_list2=info3.select('.tabBox_1')
    price=book_list2[0].find('p').get_text().strip().split('원')[0]
    line2.append({'제목':title,'저자':author,'번역':translator,'출시일':date,'쪽수':page,'가격':price})

line2

In [116]:
df2=pd.DataFrame(line2)
df2

,제목,저자,번역,출시일,쪽수,가격
0,5년 후 나에게 Q&A a Day(2022 메리골드 Edition) [양장],포터 스타일,정지현,2021년 11월 29일,368,"15,120"
1,나를 살리는 논어 한마디 : 거친 물결에 흔들리는 삶을 잡아줄 공자의 명쾌한 해답,판덩,이서연,2022년 05월 30일,320,"16,020"
2,흔한남매 10,흔한남매,-,2022년 04월 28일,192,"12,150"
3,SQL 자격검정 실전문제 : 국가공인 SQL전문가 국가공인 SQL개발자,한국데이터진흥원,-,2021년 10월 15일,279,"17,280"
4,작별인사 : 김영하 장편소설,김영하,-,2022년 05월 02일,308,"12,600"
5,"운명을 바꾸는 부동산 투자 수업: 실전편 : 내 집 마련부터 실전 아파트 투자까지,...",정태익,-,2022년 03월 22일,300,"15,120"
6,불편한 편의점(40만부 기념 벚꽃 에디션) : 김호연 장편소설,김호연,-,2021년 04월 20일,268,"12,600"
7,쥬비스 미라클 : 쥬비스 다이어트는 어떻게 신화가 되었나?,조성경,-,2022년 03월 28일,284,"14,400"
8,"운명을 바꾸는 부동산 투자 수업: 기초편 : 내 집 마련부터 실전 아파트 투자까지,...",정태익,-,2022년 03월 22일,300,"15,120"
9,나에게 고맙다(30만 부 기념 전면 개정판) : 전승환 에세이,전승환,-,2022년 02월 18일,288,"14,400"
